In [1]:
pip install tensorflow


In [2]:
import tensorflow as tf
import numpy as np

# Disable eager execution to use TF1-style code (placeholders, sessions, etc.)
tf.compat.v1.disable_eager_execution()

# ---------------------------
# 1. Load and Prepare MNIST Data
# ---------------------------
# Load the MNIST dataset (handwritten digits 0-9)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize pixel values to the range [0,1] and flatten images from 28x28 to 784 features
x_train = x_train.reshape(-1, 784).astype(np.float32) / 255.0
x_test  = x_test.reshape(-1, 784).astype(np.float32) / 255.0

# Convert labels to one-hot vectors (10 classes: digits 0-9)
num_classes = 10
y_train = np.eye(num_classes)[y_train]
y_test  = np.eye(num_classes)[y_test]

# ---------------------------
# 2. Create Placeholders for Inputs and Labels
# ---------------------------
# Placeholder for input images (batch size, 784 features)
x_ph = tf.compat.v1.placeholder(tf.float32, [None, 784], name="x")
# Placeholder for true labels (batch size, 10 classes)
y_ph = tf.compat.v1.placeholder(tf.float32, [None, 10],  name="y")

# ---------------------------
# 3. Define Network Architecture
# ---------------------------
# Hidden layer with 256 neurons
hidden_units = 256
W1 = tf.Variable(tf.random.normal([784, hidden_units], stddev=0.1), name="W1")
b1 = tf.Variable(tf.zeros([hidden_units]), name="b1")

# Output layer with 10 neurons (one per digit class)
W2 = tf.Variable(tf.random.normal([hidden_units, 10], stddev=0.1), name="W2")
b2 = tf.Variable(tf.zeros([10]), name="b2")

# ---------------------------
# 4. Build Feed-Forward Computation
# ---------------------------
# Hidden layer: Linear transformation + ReLU activation function
z1 = tf.matmul(x_ph, W1) + b1
a1 = tf.nn.relu(z1)

# Output layer: Linear transformation (logits)
logits = tf.matmul(a1, W2) + b2

# Convert logits to probabilities using softmax (for evaluation purposes)
predictions = tf.nn.softmax(logits)

# ---------------------------
# 5. Define Loss and Back-Propagation (Training)
# ---------------------------
# Compute softmax cross-entropy loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_ph, logits=logits))
# Define the Adam optimizer to minimize loss
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

# ---------------------------
# 6. Define Accuracy Metric
# ---------------------------
# Check if predicted label matches true label
correct_pred = tf.equal(tf.argmax(predictions, 1), tf.argmax(y_ph, 1))
# Compute mean accuracy
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# ---------------------------
# 7. Training Parameters
# ---------------------------
epochs = 10  # Number of training iterations
batch_size = 100  # Number of samples per batch
num_batches = x_train.shape[0] // batch_size  # Total batches per epoch

# ---------------------------
# 8. Train the Neural Network
# ---------------------------
with tf.compat.v1.Session() as sess:
    # Initialize all variables
    sess.run(tf.compat.v1.global_variables_initializer())

    for epoch in range(epochs):
        # (Optional) Shuffle training data at the start of each epoch
        indices = np.arange(x_train.shape[0])
        np.random.shuffle(indices)
        x_train = x_train[indices]
        y_train = y_train[indices]

        # Iterate through mini-batches
        for i in range(num_batches):
            batch_x = x_train[i*batch_size:(i+1)*batch_size]
            batch_y = y_train[i*batch_size:(i+1)*batch_size]
            # Perform a single optimization step
            sess.run(optimizer, feed_dict={x_ph: batch_x, y_ph: batch_y})

        # Evaluate training performance after each epoch
        train_loss, train_acc = sess.run([loss, accuracy], feed_dict={x_ph: x_train, y_ph: y_train})
        print("Epoch:", epoch+1, "Loss:", train_loss, "Accuracy:", train_acc)

    # Evaluate final performance on the test set
    test_acc = sess.run(accuracy, feed_dict={x_ph: x_test, y_ph: y_test})
    print("Test Accuracy:", test_acc)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch: 1 Loss: 0.14532256 Accuracy: 0.95875
Epoch: 2 Loss: 0.08886271 Accuracy: 0.9744167
Epoch: 3 Loss: 0.06024558 Accuracy: 0.98303336
Epoch: 4 Loss: 0.043127332 Accuracy: 0.9881667
Epoch: 5 Loss: 0.03147328 Accuracy: 0.99221665
Epoch: 6 Loss: 0.02447108 Accuracy: 0.9943
Epoch: 7 Loss: 0.023404032 Accuracy: 0.99361664
Epoch: 8 Loss: 0.01626449 Accuracy: 0.9964
Epoch: 9 Loss: 0.013151448 Accuracy: 0.99725
Epoch: 10 Loss: 0.009994991 Accuracy: 0.9978833
Test Accuracy: 0.9793
